# Figure Latitude carbon

In [ ]:
# Libraries
import os
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
import numpy as np

In [ ]:
# Directories
dir01 = '../paper_deficit/output/01_prep/'
dir04 = '../paper_deficit/output/04_out/'
dir06 = '../paper_deficit/output/06_eval/'

---

In [ ]:
# Get data
ds_agbc = xr.open_dataset(os.path.join(dir04, 'agbc.nc')) \
    .chunk(lat=5000, lon=5000)
ds_bgbc = xr.open_dataset(os.path.join(dir04, 'bgbc.nc')) \
    .chunk(lat=5000, lon=5000)
ds_soc = xr.open_dataset(os.path.join(dir04, 'soc.nc')) \
    .chunk(lat=5000, lon=5000)

ds_hilda = xr.open_mfdataset(os.path.join(dir01, 'ds_prep_hilda2015_*.zarr'),
                            engine='zarr')
ds_lesiv = xr.open_zarr(os.path.join(dir01, 'ds_prep_lesiv_nat.zarr'))

In [ ]:
# For carbon stocks plot (upper)
# extract arrays, set non-land values to nan
da_agb_pot = ds_agbc.agbc_max_prim
da_bgb_pot = ds_bgbc.bgbc_max_prim
da_soc_pot = ds_soc.soc_mean_prim

da_agb_act = ds_agbc.agbc_max_act
da_bgb_act = ds_bgbc.bgbc_max_act
da_soc_act = ds_soc.soc_mean_act

da_veg_pot = da_agb_pot + da_bgb_pot
da_veg_act = da_agb_act + da_bgb_act

# Convert to pgc and calculate sum along longitude axis
da_veg_pot_sum = (da_veg_pot * ds_agbc.area_ha * 0.000000001).sum('lon').persist()
da_soc_pot_sum = (da_soc_pot * ds_agbc.area_ha * 0.000000001).sum('lon').persist()

da_veg_act_sum = (da_veg_act * ds_agbc.area_ha * 0.000000001).sum('lon').persist()
da_soc_act_sum = (da_soc_act * ds_agbc.area_ha * 0.000000001).sum('lon').persist()

In [ ]:
# Colors
from palettable.cartocolors.qualitative import Safe_7
cpal = Safe_7

colors = [cpal.mpl_colors[i] for i in np.arange(0, 7)]

dict_colors = dict(urban = colors[0],
                   pasture = colors[1],
                   crop = colors[2],
                   foresth = colors[3],
                   forestn = colors[4],
                   shrub = colors[5],
                   other = colors[6])

c_veg_act = '#238443'
c_veg_pot = '#a1d99b'
c_soc_act = '#cc4c02'
c_soc_pot = '#fe9929'
c_edge = '#525252'

In [ ]:
# For drivers plot (lower)
# define that all forest that is not natural according to lesiv is human influenced
ds_hilda['hilda2015_forestn'] = (ds_hilda.hilda2015_forest * ds_lesiv.lesiv_nat)
ds_hilda['hilda2015_foresth'] = (ds_hilda.hilda2015_forest - ds_hilda.hilda2015_forestn)
# drop general forest class in ds_hilda
ds_hilda = ds_hilda.drop_vars('hilda2015_forest')
# for drivers plot (lower)
# calculate deficit and convert to pgc
da_veg_def = ((da_veg_pot - da_veg_act) * ds_agbc.area_ha * 0.000000001)
da_soc_def = ((da_soc_pot - da_soc_act) * ds_agbc.area_ha * 0.000000001)
# calculate sum of each hilda class along longitude axis
ds_hilda_veg_def_sum = (ds_hilda * da_veg_def).sum('lon').persist()
ds_hilda_soc_def_sum = (ds_hilda * da_soc_def).sum('lon').persist()

In [ ]:
# Prepare bar data for vegetation drivers
# (bars are plotted on top of each other)
da_veg_urban = ds_hilda_veg_def_sum \
    .to_array().sum('variable')
da_veg_pasture = ds_hilda_veg_def_sum \
    .drop_vars(['hilda2015_urban']) \
    .to_array().sum('variable')
da_veg_crop = ds_hilda_veg_def_sum \
    .drop_vars(['hilda2015_urban', 'hilda2015_pasture']) \
    .to_array().sum('variable')
da_veg_foresth = ds_hilda_veg_def_sum \
    .drop_vars(['hilda2015_urban', 'hilda2015_pasture', 'hilda2015_crop']) \
    .to_array().sum('variable')
da_veg_forestn = ds_hilda_veg_def_sum \
    .drop_vars(['hilda2015_urban', 'hilda2015_pasture', 'hilda2015_crop', 
           'hilda2015_foresth']) \
    .to_array().sum('variable')
da_veg_shrub = ds_hilda_veg_def_sum \
    .drop_vars(['hilda2015_urban', 'hilda2015_pasture', 'hilda2015_crop', 
           'hilda2015_foresth', 'hilda2015_forestn']) \
    .to_array().sum('variable')
da_veg_other = ds_hilda_veg_def_sum \
    .drop_vars(['hilda2015_urban', 'hilda2015_pasture', 'hilda2015_crop', 
           'hilda2015_foresth', 'hilda2015_forestn', 'hilda2015_shrub']) \
    .to_array().sum('variable')

In [ ]:
# Prepare bar data for soil drivers
# (bars are plotted on top of each other)
da_soc_urban = ds_hilda_soc_def_sum.to_array().sum('variable')
da_soc_pasture = ds_hilda_soc_def_sum \
    .drop_vars(['hilda2015_urban']).to_array().sum('variable')
da_soc_crop = ds_hilda_soc_def_sum \
    .drop_vars(['hilda2015_urban', 'hilda2015_pasture']).to_array().sum('variable')
da_soc_foresth = ds_hilda_soc_def_sum \
    .drop_vars(['hilda2015_urban', 'hilda2015_pasture', 'hilda2015_crop']) \
    .to_array().sum('variable')
da_soc_forestn = ds_hilda_soc_def_sum \
    .drop_vars(['hilda2015_urban', 'hilda2015_pasture', 'hilda2015_crop', 
           'hilda2015_foresth']) \
    .to_array().sum('variable')
da_soc_shrub = ds_hilda_soc_def_sum \
    .drop_vars(['hilda2015_urban', 'hilda2015_pasture', 'hilda2015_crop', 
           'hilda2015_foresth', 'hilda2015_forestn']) \
    .to_array().sum('variable')
da_soc_other = ds_hilda_soc_def_sum \
    .drop_vars(['hilda2015_urban', 'hilda2015_pasture', 'hilda2015_crop', 
           'hilda2015_foresth', 'hilda2015_forestn', 'hilda2015_shrub']) \
    .to_array().sum('variable')

In [ ]:
# Plot
fig, (ax0, ax1) = plt.subplots(figsize=(9, 6), ncols=1, nrows=2, sharex=True, dpi=600)

# Upper plot
ax0.hist(da_veg_pot_sum.lat, bins=140, weights=da_veg_pot_sum, facecolor=c_veg_pot, edgecolor=c_edge)
ax0.hist(da_veg_act_sum.lat, bins=140, weights=da_veg_act_sum, facecolor=c_veg_act, edgecolor=c_edge)
ax0.hist(da_soc_pot_sum.lat, bins=140, weights=da_soc_pot_sum *-1, facecolor=c_soc_pot, edgecolor=c_edge)
ax0.hist(da_soc_act_sum.lat, bins=140, weights=da_soc_act_sum * -1, facecolor=c_soc_act, edgecolor=c_edge)

# Legend
def prep_legend_element(color, label):
    return mpatches.Patch(label=label, facecolor=color, edgecolor=c_edge)
            #Line2D([], [], color=color, linestyle='-', linewidth=5, label=label)

            #Line2D([], [], color=c_edge, marker='o', markersize=8, 
            #      linestyle='None', linewidth=2, markerfacecolor=color, 
            #      label=label)

legend_elements_top = [prep_legend_element(c_veg_act, 'Actual AGBC + BGBC'),
                       prep_legend_element(c_veg_pot, 'Deficit AGBC + BGBC'),
                       prep_legend_element(c_soc_act, 'Actual SOC 0-30 cm'),
                       prep_legend_element(c_soc_pot, 'Deficit SOC 0-30 cm')]
leg = ax0.legend(handles=legend_elements_top, loc='upper left', ncol=1, frameon=False)
for patch in leg.get_patches():
    patch.set_height(5)
    patch.set_width(15)
    
# Lower plot
ax1.hist(ds_hilda_veg_def_sum.lat, bins=140, weights=da_veg_urban, 
        facecolor=dict_colors['urban'], edgecolor=c_edge)
ax1.hist(ds_hilda_veg_def_sum.lat, bins=140, weights=da_veg_pasture, 
        facecolor=dict_colors['pasture'], edgecolor=c_edge)
ax1.hist(ds_hilda_veg_def_sum.lat, bins=140, weights=da_veg_crop, 
        facecolor=dict_colors['crop'], edgecolor=c_edge);
ax1.hist(ds_hilda_veg_def_sum.lat, bins=140, weights=da_veg_foresth, 
        facecolor=dict_colors['foresth'], edgecolor=c_edge)
ax1.hist(ds_hilda_veg_def_sum.lat, bins=140, weights=da_veg_forestn, 
        facecolor=dict_colors['forestn'], edgecolor=c_edge)
ax1.hist(ds_hilda_veg_def_sum.lat, bins=140, weights=da_veg_shrub, 
        facecolor=dict_colors['shrub'], edgecolor=c_edge)
ax1.hist(ds_hilda_veg_def_sum.lat, bins=140, weights=da_veg_other, 
        facecolor=dict_colors['other'], edgecolor=c_edge)

ax1.hist(ds_hilda_soc_def_sum.lat, bins=140, weights=da_soc_urban *-1, 
        facecolor=dict_colors['urban'], edgecolor=c_edge)
ax1.hist(ds_hilda_soc_def_sum.lat, bins=140, weights=da_soc_pasture *-1, 
        facecolor=dict_colors['pasture'], edgecolor=c_edge)
ax1.hist(ds_hilda_soc_def_sum.lat, bins=140, weights=da_soc_crop *-1, 
        facecolor=dict_colors['crop'], edgecolor=c_edge);
ax1.hist(ds_hilda_soc_def_sum.lat, bins=140, weights=da_soc_foresth *-1, 
        facecolor=dict_colors['foresth'], edgecolor=c_edge)
ax1.hist(ds_hilda_soc_def_sum.lat, bins=140, weights=da_soc_forestn *-1, 
        facecolor=dict_colors['forestn'], edgecolor=c_edge)
ax1.hist(ds_hilda_soc_def_sum.lat, bins=140, weights=da_soc_shrub *-1, 
        facecolor=dict_colors['shrub'], edgecolor=c_edge)
ax1.hist(ds_hilda_soc_def_sum.lat, bins=140, weights=da_soc_other *-1, 
        facecolor=dict_colors['other'], edgecolor=c_edge);

# Legend
legend_elements_bottom = [prep_legend_element(dict_colors['urban'], 'Urban'),
                          prep_legend_element(dict_colors['pasture'], 'Pasture'),
                          prep_legend_element(dict_colors['crop'], 'Cropland'),
                          prep_legend_element(dict_colors['foresth'], 'Managed forest'),
                          prep_legend_element(dict_colors['forestn'], 'Natural forest'),
                          prep_legend_element(dict_colors['shrub'], 'Shrubland'),
                          prep_legend_element(dict_colors['other'], 'Other land')]

leg = ax1.legend(handles=legend_elements_bottom, loc='upper left', ncol=1, frameon=False)
for patch in leg.get_patches():
    patch.set_height(5)
    patch.set_width(15)

# Plot settings
ax0.hlines(0, -60, 80, colors='#000000', linewidth = 2)   
ax0.set_ylabel('Carbon stocks (Pg C)')    

ax1.hlines(0, -60, 80, colors='#000000', linewidth = 2)   
ax1.set_ylabel('Carbon deficit (Pg C)')    

ax1.set_xlabel('Latitude')
ax1.set_xlim(-60, 80)

ax0.text(77, 2, 'Vegetation', va='bottom', fontsize='large', weight='bold', rotation=270)
ax0.text(77, -2, 'Soil', va='top', fontsize='large', weight='bold', rotation=270)

ax1.text(77, 0.4, 'Vegetation', va='bottom', fontsize='large', weight='bold', rotation=270)
ax1.text(77, -0.4, 'Soil', va='top', fontsize='large', weight='bold', rotation=270)

linewidth=2

ax0.spines['right'].set_visible(False) # remove right plot boundary
ax0.spines['top'].set_visible(False) # remove top plot boundary
ax0.spines['left'].set_linewidth(linewidth) # make left axis thicker
ax0.spines['bottom'].set_visible(False) # make bottom axis thicker
ax0.xaxis.set_visible(False)
ax0.yaxis.set_tick_params(width=linewidth) # make y-axis tick marks thicker
ax0.set_yticks([20, 15, 10, 5, 0, -5, -10, -15])
ax0.set_yticklabels([20, 15, 10, 5, 0, 5, 10, 15])

ax1.spines['right'].set_visible(False) # remove right plot boundary
ax1.spines['top'].set_visible(False) # remove top plot boundary
ax1.spines['left'].set_linewidth(linewidth) # make left axis thicker
ax1.spines['bottom'].set_linewidth(linewidth) # make bottom axis thicker
ax1.xaxis.set_tick_params(width=linewidth) # make x-axis tick marks thicker
ax1.yaxis.set_tick_params(width=linewidth) # make y-axis tick marks thicker
ax1.set_yticks([5, 4, 3, 2, 1, 0, -1])
ax1.set_yticklabels([5, 4, 3, 2, 1, 0, 1])

# Plot settings
plt.figtext(0.01, 0.95, 'A', fontsize='x-large', weight='bold')
plt.figtext(0.01, 0.495, 'B', fontsize='x-large', weight='bold');

fig.tight_layout()

# Export
plt.savefig(os.path.join(dir06, f'pdf/figs09_latitude_carbon.pdf'), bbox_inches='tight', dpi=600)
plt.savefig(os.path.join(dir06, f'png/figs09_latitude_carbon.png'), bbox_inches='tight', dpi=600);